In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 6
height = 5
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 180

puzzle.naming = {
    None: '  ',
    'e': ' _',
    'p': ' +',
    'm': ' -',
}
for i in range(9):
    puzzle.naming[str(i)] = f" {i}"

puzzle.latex_naming = {
    None: ' ',
    'e': ' ',
    'p': '\\signmine{\\ensuremath{+}}',
    'm': '\\signmine{\\ensuremath{-}}',
    'diff': '{\\scalebox{1.0}{$\\Delta$}}',
}
for i in range(0, 10):
    puzzle.latex_naming[str(i)] = f'\\Large \\ensuremath{{{i}}}'
    puzzle.latex_naming[str(-i)] = f'\\Large \\ensuremath{{{-i}}}'
    
puzzle.domain_program += f"""
    value(m;p).
    value(N) :- num(N).
    num(0..8).
    num(-N) :- num(N).
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : cell(C), value(V) }.
    :- puzzle(C,m), cell(C).
    :- puzzle(C,p), cell(C).
    :- puzzle(C,0), cell(C).
    
    1 { guessed_number(diff,1) } 1.
"""
puzzle.puzzle_constraints_program += enc_library['diagonally_adjacent_cells']
puzzle.puzzle_constraints_program += """
    #minimize { 1@10,puzzle(C,V) : puzzle(C,V) }.

    mixed_cell(C) :- solution(C,N), num(N),
        ring_around_cell(C,C1,1), solution(C1,m),
        ring_around_cell(C,C2,1), solution(C2,p).
    :- not 2 { mixed_cell(C) }.
    #maximize { 1@20,mixed_cell(C) : mixed_cell(C) }.
        
    :- puzzle(C,V) : solution(C,V), num(V), V > 0.
    
    % TO INCREASE DIFFICULTY
    puzzle(C) :- puzzle(C,_).
    :- puzzle(C,V), num(V),
        |V| = #count { D : cell(D), ring_around_cell(C,D,1), not puzzle(D) }.
    
    % TO INCREASE DIFFICULTY
    :- puzzle(C1,V1), puzzle(C2,V2), diagonally_adjacent_cells(C1,C2).
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['ring_around_cell']
puzzle.solution_program += enc_library['adjacent_cells']
puzzle.solution_program += enc_library['diagonally_adjacent_cells']
puzzle.solution_program += """
    :- cell(C), solution(C,N), num(N),
        P = #count { D : cell(D), ring_around_cell(C,D,1), solution(D,p) },
        M = #count { D : cell(D), ring_around_cell(C,D,1), solution(D,m) },
        N != P - M.
"""

puzzle.essential_solution_constraints = [
    """
    :- cell(C1), cell(C2), adjacent_cells(C1,C2),
        solution(C1,p), solution(C2,m).
    """,
    """
    :- cell(C1), cell(C2), diagonally_adjacent_cells(C1,C2),
        solution(C1,p), solution(C2,p).
    :- cell(C1), cell(C2), diagonally_adjacent_cells(C1,C2),
        solution(C1,m), solution(C2,m).
    """,
    """
    :- P = #count { C : cell(C), solution(C,p) },
        M = #count { C : cell(C), solution(C,m) },
        guessed_number(diff,D),
        |P-M| != D.
    """,
]

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
.......
Stopped after solving timeout..
Solving time: 223.92 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

|  |  |  |  |-1|  |
|  |  |  |  |  |  |
| 1|  | 2| 5|  |  |
|  |  |  |  |  |  |
|  |  | 1| 3|  | 1|

diff = 1


In [5]:
print(puzzle.pretty_repr_solution())

| 0| -|-1| -|-1| -|
| +| 0| +| 1| +|-1|
| 1| -| 2| 5| 2| -|
| +| 0| +| +| +| 0|
| 0| -| 1| 3| 2| 1|


In [6]:
print(puzzle.latex_repr_puzzle())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,5);
    \draw[ultra thick]
      (0,0) rectangle (6,5);
      \node[anchor=center] (1;1) at (0.5,4.5) { };
      \node[anchor=center] (1;2) at (1.5,4.5) { };
      \node[anchor=center] (1;3) at (2.5,4.5) { };
      \node[anchor=center] (1;4) at (3.5,4.5) { };
      \node[anchor=center] (1;5) at (4.5,4.5) {\Large \ensuremath{-1}};
      \node[anchor=center] (1;6) at (5.5,4.5) { };
      \node[anchor=center] (2;1) at (0.5,3.5) { };
      \node[anchor=center] (2;2) at (1.5,3.5) { };
      \node[anchor=center] (2;3) at (2.5,3.5) { };
      \node[anchor=center] (2;4) at (3.5,3.5) { };
      \node[anchor=center] (2;5) at (4.5,3.5) { };
      \node[anchor=center] (2;6) at (5.5,3.5) { };
      \node[anchor=center] (3;1) at (0.5,2.5) {\Large \ensuremath{1}};
      \node[anchor=center] (3;2) at (1.5,2.5) { };
      \node[anchor=center] (3;3) at (2.5,2.5) {\Large \ensuremath{2}};
      \node[anchor=center] (3;4) at (3.5,2.5) {\

In [7]:
print(puzzle.latex_repr_solution())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,5);
    \draw[ultra thick]
      (0,0) rectangle (6,5);
      \node[anchor=center] (1;1) at (0.5,4.5) {\Large \ensuremath{0}};
      \node[anchor=center] (1;2) at (1.5,4.5) {\signmine{\ensuremath{-}}};
      \node[anchor=center] (1;3) at (2.5,4.5) {\Large \ensuremath{-1}};
      \node[anchor=center] (1;4) at (3.5,4.5) {\signmine{\ensuremath{-}}};
      \node[anchor=center] (1;5) at (4.5,4.5) {\Large \ensuremath{-1}};
      \node[anchor=center] (1;6) at (5.5,4.5) {\signmine{\ensuremath{-}}};
      \node[anchor=center] (2;1) at (0.5,3.5) {\signmine{\ensuremath{+}}};
      \node[anchor=center] (2;2) at (1.5,3.5) {\Large \ensuremath{0}};
      \node[anchor=center] (2;3) at (2.5,3.5) {\signmine{\ensuremath{+}}};
      \node[anchor=center] (2;4) at (3.5,3.5) {\Large \ensuremath{1}};
      \node[anchor=center] (2;5) at (4.5,3.5) {\signmine{\ensuremath{+}}};
      \node[anchor=center] (2;6) at (5.5,3.5) {\Large \ensuremath{-

In [8]:
latex_preamble = """
    \makeatletter
    \newcommand*\signmine[2][1.6]{\tikz[baseline=(char.base)]{
        \node[shape=circle, fill=black!30, inner sep=0pt, 
            minimum height={\f@size*#1},] (char) {\vphantom{WAH1g}#2};}}
    \makeatother
"""